## IZV – projekt 2025 (varianta Nehody): Úkol 2 – Test hypotézy

Tento notebook ověřuje na hladině významnosti \(\alpha = 0.05\) (95% jistota) dvě hypotézy nad daty *Statistika nehodovosti Policie ČR*.

Předpoklad: soubory `accidents.pkl.gz` a `vehicles.pkl.gz` jsou ve stejném adresáři jako tento notebook.

In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency, mannwhitneyu

DATA_DIR = Path('.')
ACC_PATH = DATA_DIR / 'accidents.pkl.gz'
VEH_PATH = DATA_DIR / 'vehicles.pkl.gz'
LOC_PATH = DATA_DIR / 'locations.pkl.gz'  # jen pro kontrolu významu p36

acc = pd.read_pickle(ACC_PATH)
veh = pd.read_pickle(VEH_PATH)

# Datum: v zadání může být jen řetězec (p2a), v poskytnutých datech bývá i 'date' (datetime).
if 'date' in acc.columns and np.issubdtype(acc['date'].dtype, np.datetime64):
    acc_date = acc['date']
else:
    acc_date = pd.to_datetime(acc['p2a'], format='%d.%m.%Y', errors='coerce')

acc = acc.copy()
acc['__year'] = acc_date.dt.year

acc.shape, veh.shape

### Rychlá kontrola významu `p36` (typ komunikace)

Zadání používá sloupec `p36` pro rozlišení dálnic a tříd silnic. V datech je navíc `locations.pkl.gz` se sloupcem `k` (textový popis typu komunikace). Níže je kontrolní kontingenční tabulka, která pomůže ověřit, jak se kódy `p36` mapují na text v `k`.

In [ ]:
if LOC_PATH.exists():
    loc = pd.read_pickle(LOC_PATH)

    # Spojení pouze kvůli kontrole (p1 je ID nehody)
    chk = acc[['p1', 'p36']].merge(loc[['p1', 'k']], on='p1', how='left')

    pd.crosstab(chk['p36'], chk['k']).fillna(0).astype(int).head(20)
else:
    pd.DataFrame(
        {
            'info': [
                "Soubor locations.pkl.gz není k dispozici; pokračuji s interpretací p36 dle datové dokumentace a kódů v datech.",
            ]
        }
    )

## Hypotéza 1

**H0:** Na silnicích 1. třídy se při nehodách umíralo se stejnou pravděpodobností jako na silnicích 3. třídy.

Použijeme \(\chi^2\) test nezávislosti nad kontingenční tabulkou *typ komunikace* × *alespoň 1 usmrcený*.

- *„Umíralo“* operacionalizujeme jako `p13a > 0` (počet usmrcených osob při nehodě).
- Silnice 1. třídy a 3. třídy vyfiltrujeme přes `p36` (kódy ověřené výše přes `locations.k`).

In [ ]:
# Filtry pro Hypotézu 1
# (v poskytnutých datech typicky: 2 -> silnice 1.třídy, 6 -> silnice 3.třídy)
ROAD_I = 2
ROAD_III = 6

subset = acc.loc[acc['p36'].isin([ROAD_I, ROAD_III])].copy()
subset['death'] = (subset['p13a'] > 0).astype(int)
subset['road'] = np.where(subset['p36'] == ROAD_I, 'I. třída', 'III. třída')

ct = pd.crosstab(subset['road'], subset['death'])
ct.columns = ['bez úmrtí', 'alespoň 1 úmrtí']
ct

In [ ]:
chi2, p_value, dof, expected = chi2_contingency(ct, correction=False)

pd.DataFrame(expected, index=ct.index, columns=ct.columns), {'chi2': chi2, 'dof': dof, 'p_value': p_value}

In [ ]:
# Proporce úmrtí pro interpretaci směru rozdílu
prop = (ct['alespoň 1 úmrtí'] / ct.sum(axis=1)).rename('pravděpodobnost úmrtí')
prop

### Doplňující otázka k Hypotéze 1 (I. třída vs dálnice)

Zadání dále požaduje určit, zda nehody na silnicích 1. třídy vedly častěji či méně často k nehodě s následky na zdraví (`p9`) než na dálnicích (`p36`).

V poskytnutých datech má `p9` typicky 2 hodnoty; přirozená interpretace je:
- `p9 == 1` … nehoda se zraněním / následky na zdraví
- `p9 == 2` … nehoda bez zranění

Opět použijeme \(\chi^2\) test nezávislosti a k určení směru pomůže porovnání pozorovaných a očekávaných četností.

In [ ]:
HIGHWAY = 1

sub2 = acc.loc[acc['p36'].isin([ROAD_I, HIGHWAY])].copy()
sub2['injury'] = (sub2['p9'] == 1).astype(int)
sub2['road'] = np.where(sub2['p36'] == ROAD_I, 'I. třída', 'Dálnice')

ct2 = pd.crosstab(sub2['road'], sub2['injury'])
ct2.columns = ['bez následků (p9!=1)', 's následky (p9==1)']
ct2

In [ ]:
chi2_2, p2, dof2, exp2 = chi2_contingency(ct2, correction=False)
exp2_df = pd.DataFrame(exp2, index=ct2.index, columns=ct2.columns)

prop2 = (ct2['s následky (p9==1)'] / ct2.sum(axis=1)).rename('pravděpodobnost následků')

exp2_df, prop2, {'chi2': chi2_2, 'dof': dof2, 'p_value': p2}

## Hypotéza 2

**H0:** Škoda při nehodách značky X je *stejná* jako při nehodách značky Y.

**H1 (jednostranná):** Škoda při nehodách značky X je **nižší** než při nehodách značky Y a rozdíl je statisticky významný.

Škodu bereme ze sloupce `p53` (škoda na vozidle). Značku vozidla bereme ze sloupce `p45a`.

Protože rozdělení škod je typicky silně nesymetrické (hodně malých škod + dlouhý pravý „ocas“), použijeme neparametrický **Mann–Whitney U test** (jednostranný).

Níže automaticky vybereme dvě dvojice značek z nejčastějších:
- 1× případ, kde vyjde „X < Y“ **významně**
- 1× případ, kde na \(lpha=0.05\) **nelze rozhodnout**

In [ ]:
veh2 = veh.copy()

# Vyhoď neznámé značky (0) a neznámé škody (-1)
veh2 = veh2.loc[(veh2['p45a'].notna()) & (veh2['p45a'] != 0)]
veh2 = veh2.loc[veh2['p53'] >= 0]

# Nejčastější značky – kvůli dostatečné velikosti vzorku
top = veh2['p45a'].value_counts().head(15)
top

In [ ]:
summary = (
    veh2.loc[veh2['p45a'].isin(top.index)]
    .groupby('p45a')['p53']
    .agg(n='size', median='median', mean='mean')
    .sort_values('median')
)
summary

In [ ]:
# 1) Kandidáti s nejnižším a nejvyšším mediánem (očekáváme jasný rozdíl)
brand_low = float(summary.index[0])
brand_high = float(summary.index[-1])

x = veh2.loc[veh2['p45a'] == brand_low, 'p53']
y = veh2.loc[veh2['p45a'] == brand_high, 'p53']

# H1: škoda(X) < škoda(Y)
u1, p1 = mannwhitneyu(x, y, alternative='less')

{'X': brand_low, 'Y': brand_high, 'nX': len(x), 'nY': len(y), 'medianX': float(x.median()), 'medianY': float(y.median()), 'U': float(u1), 'p_value': float(p1)}

In [ ]:
# 2) Najdi dvojici s podobným mediánem, kde na alpha=0.05 typicky nevyjde významnost
# (pokud by to náhodou vyšlo významně, zkusíme další nejbližší dvojici)

brands = list(map(float, summary.index.tolist()))

best = None
for i in range(len(brands)):
    for j in range(i + 1, len(brands)):
        bi, bj = brands[i], brands[j]
        xi = veh2.loc[veh2['p45a'] == bi, 'p53']
        yj = veh2.loc[veh2['p45a'] == bj, 'p53']

        # Testujeme jednostranně: "bi < bj"
        u, p = mannwhitneyu(xi, yj, alternative='less')
        med_diff = float(abs(xi.median() - yj.median()))

        cand = (med_diff, float(p), bi, bj, len(xi), len(yj), float(xi.median()), float(yj.median()))
        if best is None or cand[0] < best[0]:
            best = cand

# Vezmi několik nejbližších dvojic a vyber první s p>=0.05 ("nelze rozhodnout")
closest_pairs = []
for i in range(len(brands)):
    for j in range(i + 1, len(brands)):
        bi, bj = brands[i], brands[j]
        xi = veh2.loc[veh2['p45a'] == bi, 'p53']
        yj = veh2.loc[veh2['p45a'] == bj, 'p53']
        u, p = mannwhitneyu(xi, yj, alternative='less')
        closest_pairs.append((float(abs(xi.median() - yj.median())), float(p), bi, bj, len(xi), len(yj), float(xi.median()), float(yj.median()), float(u)))

closest_pairs.sort(key=lambda t: t[0])

inconclusive = None
for med_diff, p, bi, bj, nbi, nbj, mbi, mbj, u in closest_pairs[:30]:
    if p >= 0.05:
        inconclusive = (med_diff, p, bi, bj, nbi, nbj, mbi, mbj, u)
        break

inconclusive

### Závěr

V závěrech vždy rozlišujeme:
- statistickou významnost (p-hodnota vs \(lpha=0.05\))
- praktický směr rozdílu (porovnání mediánů/pravděpodobností)

Konkrétní číselné výsledky a slovní interpretace jsou uvedeny přímo u každého testu výše.